In [ ]:
import os, sys
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
#import gdal
from osgeo import gdal, gdalnumeric, ogr, osr
from PIL import Image, ImageDraw
import pyproj 

gdal.UseExceptions()

In [ ]:
SOIL_DIR='/prj/GREENHOUSE/SOIL_PROPERTIES/datasets/England_Wales_Soil_Data/LDE16_12_SRUC_Tarsitano/Spatial Soil Data/'
shp_file=SOIL_DIR+'NATMAPtopsoiltexture.shp'

raster_fn=SOIL_DIR+'NATMAPtopsoiltexture.tif'
LatLon_file='/users/eow/edwcom/CHESS/chess_landcover_2000.nc'



In [ ]:

driverName='ESRI Shapefile'
drv=ogr.GetDriverByName(driverName)
dataSource=drv.Open(shp_file)

Layer=dataSource.GetLayer()


In [ ]:
source_srs = Layer.GetSpatialRef()
print(source_srs)
x_min,x_max,y_min,y_max=Layer.GetExtent()
print(x_min,x_max,y_min,y_max)

In [ ]:
NoData_value=-9999

# Create the destination data source
pixel_size=25
x_res = int((x_max - x_min) / pixel_size)
y_res = int((y_max - y_min) / pixel_size)
target_ds = gdal.GetDriverByName('GTiff').Create(raster_fn, x_res, y_res, gdal.GDT_Byte)
target_ds.SetGeoTransform((x_min, pixel_size, 0, y_max, 0, -pixel_size))
band = target_ds.GetRasterBand(1)
band.SetNoDataValue(NoData_value)


In [ ]:
# Rasterize
gdal.RasterizeLayer(target_ds, [1], Layer, burn_values=[0])


In [ ]:
osgb36=pyproj.Proj('+proj=tmerc +lat_0=49 +lon_0=-2 +k=0.9996012717 +x_0=400000 +y_0=-100000 +ellps=airy +datum=OSGB36 +units=m +no_defs' ) # UK Ordnance Survey, 1936 datum 



In [ ]:
# Read LatLon file
LLinf=nc.Dataset(LatLon_file,'r')
lats=LLinf.variables['lat'][:]
lons=LLinf.variables['lon'][:]
LLinf.close()